In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("..")
sys.path.append("../../")

import armControl

In [4]:
myArm = armControl.remoteRobotArm()
armTheta_motorTheta = myArm.armTheta_motorTheta #calculates motor theta given arm theta -->backward
motorTheta_armTheta_full = myArm.motorTheta_armTheta_full #calculates arm theta given motor theta -->forward



In [37]:
import numpy as np
from sklearn.utils.extmath import cartesian

class configGenerator_VREP(object):
    def generate_sample(config = 2):
        # prepare training set
#        joint_range_lower_limit = np.array([-2e-1, -4.500e-2, -9.000e+1, -5.200e+1, -5.020e+1, -5.200e+1, 0])
#        joint_range = np.array([4e-1, 0.95e0, 1.800e+2, 1.040e+2, 1.020e+2, 1.040e+2, 5.500e-2])

        #units are radians for revolute joints: http://www.coppeliarobotics.com/helpFiles/en/jointDescription.htm
        joint_range_lower_limit = np.array([-0, -0, -0, -100, -100, -100, -1.5e-2])
        joint_range_lower_limit[2:-1] = joint_range_lower_limit[2:-1] * np.pi/180
        joint_range = np.array([0, 0, 0, 200, 200, 200, 3e-2])
        joint_range[2:-1] = joint_range[2:-1] * np.pi/180
        joint_range_upper_limit = joint_range_lower_limit + joint_range
        
        print('joint lower limit\n', joint_range_lower_limit)
        print('joint upper limit\n', joint_range_upper_limit)
        
        sample = (np.linspace(0,1,config)[:,None]*(joint_range_upper_limit
                               -joint_range_lower_limit)+joint_range_lower_limit)
        
        #x1, x2 = np.meshgrid(sample_j1, sample_j2)
        #t = np.array(np.meshgrid(sample[0,:],sample[1,:])).T.reshape(-1,7)
#        sample0 = ((np.linspace(0,1,config[0])[:,None]*(joint_range_upper_limit
#                           -joint_range_lower_limit)+joint_range_lower_limit))
#        sample1 = ((np.linspace(0,1,config[1])[:,None]*(joint_range_upper_limit
#                           -joint_range_lower_limit)+joint_range_lower_limit))
#        sample2 = ((np.linspace(0,1,config[2])[:,None]*(joint_range_upper_limit
#                           -joint_range_lower_limit)+joint_range_lower_limit))
#        sample3 = ((np.linspace(0,1,config[3])[:,None]*(joint_range_upper_limit
#                           -joint_range_lower_limit)+joint_range_lower_limit))
#        sample4 = ((np.linspace(0,1,config[4])[:,None]*(joint_range_upper_limit
#                           -joint_range_lower_limit)+joint_range_lower_limit))
#        sample5 = ((np.linspace(0,1,config[5])[:,None]*(joint_range_upper_limit
#                           -joint_range_lower_limit)+joint_range_lower_limit))
#        
#        #x1, x2 = np.meshgrid(sample_j1, sample_j2)
#        #t = np.array(np.meshgrid(sample[0,:],sample[1,:])).T.reshape(-1,7)
#        sample = np.vstack((sample0,sample1,sample2,sample3,sample4,sample5))
        
        total_sample = cartesian((sample[:, 0], sample[:, 1], sample[:, 2], sample[:, 3], 
                       sample[:, 4], sample[:, 5], sample[:, 6]))
        return total_sample


In [40]:
config = configGenerator_VREP.generate_sample(13)  # set sample number here
total_config = config.shape[0]
print("total configs = ", total_config)

joint lower limit
 [ 0.          0.          0.         -1.74532925 -1.74532925 -1.74532925
 -0.015     ]
joint upper limit
 [0.         0.         0.         1.74532925 1.74532925 1.74532925
 0.015     ]
total configs =  35831808


In [42]:
motor_angles = np.zeros(config.shape)
for i in range(config.shape[0]):
    motor_angles[i,:] = armTheta_motorTheta@config[i,:]
    
print('Maximum motor angles in degrees: {}'.format(np.abs(motor_angles).max(axis=0)/np.pi*180))

Maximum motor angles in degrees: [  0.           0.           0.         220.47244094 393.7007874
 425.19685039 378.08810027]
